In [79]:
import pandas as pd, numpy as np, re, nltk, math
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

In [ ]:
"""
Stopwords são palavras como artigos e etc que não tem que entrar quando estamos fazendo o projeto de previsão
ou seja essa stopwords é para retirar essas palavras da string que estaremos analizando
"""
nltk.download('stopwords') 
stopwords.words('portuguese')

## Ingestão Base de treinamento e tratamento

Base retirada do projeto Fake.br-corpus, [Fake.br-Corpus/preprocessed/pre-processed.csv](https://github.com/roneysco/Fake.br-Corpus/tree/master/preprocessed)

In [52]:
'''
Ingestão Base de treinamento e tratamento

Base retirada do projeto Fake.br-corpus, [Fake.br-Corpus/preprocessed/pre-processed.csv](https://github.com/roneysco/Fake.br-Corpus/tree/master/preprocessed)

'''
df_nlp_ru_3362654 = pd.read_csv('pre-processed.csv')

In [53]:
# Tratando a base e criando um label com informaçoes de 1 e 0, 1 para fakenews e 0 para não fakenews
df_nlp_ru_3362654.rename(columns={'label': 'status'}, inplace=True)
df_nlp_ru_3362654['label'] = df_nlp_ru_3362654['status'].map({'fake': 1, 'TRUE': 0}, na_action=None)
df_nlp_ru_3362654 = df_nlp_ru_3362654[['index', 'label', 'status', 'preprocessed_news']]

In [55]:
#Verificando se há nulos:

"""
Observamos que não há nullo, é extremamente importante realizar esse ponto para não termos ruído nos nossos dados
"""
df_nlp_ru_3362654.isnull().sum()

index                0
label                0
status               0
preprocessed_news    0
dtype: int64

In [56]:
df_nlp_ru_3362654

,index,label,status,preprocessed_news
0,0,1,fake,katia abreu diz vai colocar expulsao moldura n...
1,1,1,fake,ray peita bolsonaro conservador fake entrevist...
2,2,1,fake,reinaldo azevedo desmascarado policia federal ...
3,3,1,fake,relatorio assustador bndes mostra dinheiro pub...
4,4,1,fake,radialista americano fala sobre pt vendem ilus...
...,...,...,...,...
7195,7195,0,TRUE,jornal britanico acao contra lula lava jato se...
7196,7196,0,TRUE,temer diz acionou pf cade investigar aumentos ...
7197,7197,0,TRUE,obstaculos politicos temer especialistas ouvid...
7198,7198,0,TRUE,setembro boa noite aqui estao principais notic...


In [57]:
"""
Essa função tenta trazer até a palavra raiz dela exemplo atuando, ele levará ate atuar ou seja no menor nível que a palavra pode chegar
"""

# Criar função de steaming

porter_stemmer = PorterStemmer()

def steaming_tokenizer(dataframe):
        # todas que começar de A a Z maisculo e minusculo para manter
        words = re.sub(r"[^A-Za-z]", " ", dataframe).lower().split()
        #pegar que não esteja na stopwords e reduza a menor parte dela
        words = [porter_stemmer.stem(word) for word in words if not words in stopwords.words('portuguese')] 
        words = ' '.join(words)
        return words

In [58]:
#Agora vamos aplicar o processamento do tokenizer para as regras da função acima esse processo demora bastante como é um 
#trabalho academico da não há necessidade de um processamento paralelo demorou 50min para processar
df_nlp_ru_3362654['features'] = df_nlp_ru_3362654['preprocessed_news'].apply(steaming_tokenizer)

In [59]:
#Quando printamos podemos observar que reduzimos a menor nivel da palavra e que removemos alguns pontos dentro da stowords
print(df_nlp_ru_3362654['features'])

0       katia abreu diz vai colocar expulsao moldura n...
1       ray peita bolsonaro conservador fake entrevist...
2       reinaldo azevedo desmascarado policia feder fe...
3       relatorio assustador bnde mostra dinheiro publ...
4       radialista americano fala sobr pt vendem ilusa...
                              ...                        
7195    jornal britanico acao contra lula lava jato se...
7196    temer diz acion pf cade investigar aumento pre...
7197    obstaculo politico temer especialista ouvido a...
7198    setembro boa noit aqui estao principai noticia...
7199    envolvo politica diz brasileiro preso venezuel...
Name: features, Length: 7200, dtype: object


In [60]:
#Separando em Features e Lavels pegando esses valores e jogando em variáveis X e Y
X, Y = df_nlp_ru_3362654['features'].values, df_nlp_ru_3362654['label'].values

In [61]:
#Realizando a conversão de textual para númerico
# Pois o modelo não está preparado para receber várias palavras ele é preparado para trabalhar com números
# Então usaremos TF-IDF que irá realizar a contagem de palavras e verificar as que ocorrem mais e as que ocorrem menos
# Lembdando que esses números vem já normalizados
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [81]:
names = vectorizer.get_feature_names()

c:\Users\lourenpe\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [62]:
# Treinando com 80% do dataset quando seto test_size(0.2), stratify para balancear as classes de teste

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size= 0.2, stratify=Y, random_state=42) 

In [84]:
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(5760, 67043) (5760,)
(1440, 67043) (1440,)


In [83]:
print(X_train)

  (0, 65888)	0.07315316097990515
  (0, 65834)	0.06019306416347457
  (0, 65795)	0.019643256610796335
  (0, 65754)	0.07468135723116504
  (0, 65750)	0.021385820661590917
  (0, 65421)	0.03187658878864742
  (0, 65268)	0.050834493262129264
  (0, 65222)	0.0432544948246975
  (0, 65082)	0.03850056655063685
  (0, 64265)	0.03414130630637563
  (0, 64217)	0.02898214930151291
  (0, 64083)	0.01924063077432259
  (0, 64064)	0.04178670156479355
  (0, 63753)	0.10693178728728939
  (0, 63334)	0.030580716725002257
  (0, 63316)	0.07380925620157146
  (0, 63185)	0.18383007499278334
  (0, 62750)	0.032586750341521864
  (0, 62440)	0.02543814580067296
  (0, 62318)	0.023130811867105635
  (0, 62313)	0.014318835187829558
  (0, 62163)	0.03545070891170356
  (0, 61810)	0.021922121024726696
  (0, 61572)	0.04582445595554847
  (0, 61429)	0.02497489199721214
  :	:
  (5759, 28230)	0.1321391299885641
  (5759, 27398)	0.05830227314941009
  (5759, 27322)	0.07070306179427983
  (5759, 24302)	0.049819797289676775
  (5759, 21195)	0.

In [66]:
#Treinando o modelo com logistic
# O Logistic regression é um modelo de classificação binária, ou seja como nossa base é binária ou é 0 ou é 1, o logistic regression é o mais 
#viável de utilizarmos
lr = LogisticRegression()
lr.fit(X_train, Y_train)
hist = lr.fit(X_train, Y_train)

In [67]:
# Previsão 
pred = lr.predict(X_test)

In [68]:
#Verificando a acuracia dos testes, Ou seja sempre que quisermos testar podemos pegar pedaços de teste diferentes para visualizar se está acertando ou não
acc = accuracy_score(Y_test, pred)
f'Acurácia: {acc * 100:.2f}'

'Acurácia: 94.79'

In [71]:
plt.figure()
plt.plot(hist.history['accuracy'], lw=2.0, color='b', label='train')
plt.plot(hist.history['val_accuracy'], lw=2.0, color='r', label='val')

AttributeError: 'LogisticRegression' object has no attribute 'history'

<Figure size 432x288 with 0 Axes>